In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import sklearn.metrics as metrics

In [2]:
# Source: 
# https://www.learnopencv.com/image-classification-using-transfer-learning-in-pytorch/

# Applying Transforms to the Data
image_transforms = {
    'train': transforms.Compose([
#         transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
#         transforms.RandomRotation(degrees=15),
#         transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


# Set train and valid directory paths
train_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/train'
valid_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/valid'
test_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/test'
 
# Batch size
bs = 64
 
# Number of classes
num_classes = 2
 
# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
    'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
}
 
# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])
 
# Create iterators for the Data loaded using DataLoader module
train_data = torch.utils.data.DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = torch.utils.data.DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data = torch.utils.data.DataLoader(data['test'], batch_size=bs, shuffle=True)
 
# Print the train, validation and test set data sizes
train_data_size, valid_data_size, test_data_size

(151293, 35876, 35659)

In [3]:
# Load pretrained AlexNet Model
alexnet = torchvision.models.alexnet(pretrained=True)

In [4]:
# Source: 
# https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/

# Updating the second classifier
alexnet.classifier[4] = nn.Linear(4096,1024)

#Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
# AdVo: i will have a binary classification , thus only 2 output nodes
alexnet.classifier[6] = nn.Linear(1024, num_classes)

Define loss function: Cross Entropy Loss

Note: Improvement option by adding weight class check

In [5]:
# Source: 
# https://github.com/choosehappy/PytorchDigitalPathology/blob/master/visualization_densenet/train_densenet.ipynb

# we have the ability to weight individual classes, in this case we'll do so based on their presense in the trainingset
# to avoid biasing any particular class
# nclasses = dataset["train"].classsizes.shape[0]
# class_weight=dataset["train"].classsizes
# class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor').to(device)

# print(class_weight) #show final used weights, make sure that they're reasonable before continouing


# criterion = torch.nn.CrossEntropyLoss(weight = class_weight)
criterion = nn.CrossEntropyLoss()


Define optimizer: Adam

In [6]:
# Source: 
# https://github.com/choosehappy/PytorchDigitalPathology/blob/master/visualization_densenet/train_densenet.ipynb

# adam is going to be the most robust, though perhaps not the best performing, typically a good place to start
optimizer = optim.Adam(alexnet.parameters()) 

In [7]:
# for epoch in range(10):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i, data in enumerate(train_data, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data[0].to(device), data[1].to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         output = alexnet(inputs)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

# print('Finished Training of AlexNet')

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
history = []
epochs = 5
for epoch in range(epochs):
    epoch_start = time.time()
#     print("Epoch: {}/{}".format(epoch, epochs))
     
    # Set to training mode
    alexnet.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    # Iterate through all batches of training data
    for i, (inputs, labels) in enumerate(train_data):
 
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        # Clean existing gradients
        optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = alexnet(inputs)
        
#         print("predictions")
#         print(outputs)
         
        # Compute loss
        loss = criterion(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
#         print("ret")
#         print(ret)
#         print("predictions")
#         print(predictions)

        correct_counts = predictions.eq(labels.data.view_as(predictions))
#         print("correct counts")
#         print(correct_counts)
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
#         print("acc")
#         print(acc)
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)
         
#         print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))
        

    # Validation is carried out in each epoch immediately after the training loop
    # Validation - No gradient tracking needed
    with torch.no_grad():

        # Set to evaluation mode
        alexnet.eval()

        # Validation loop
        # Iterate through all batches of validation data
        for j, (inputs, labels) in enumerate(valid_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = alexnet(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            valid_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to valid_acc
            valid_acc += acc.item() * inputs.size(0)

#             print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

    # Find average training loss and training accuracy
    avg_train_loss = train_loss/train_data_size
    avg_train_acc = train_acc/float(train_data_size)

    # Find average training loss and training accuracy
    avg_valid_loss = valid_loss/valid_data_size
    avg_valid_acc = valid_acc/float(valid_data_size)

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

    epoch_end = time.time()

    print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, \n Time (train+val): {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

    # Stop "epoch" for
#     break

Epoch : 000, Training: Loss: 0.6719, Accuracy: 61.6169%, 
		Validation : Loss : 0.6640, Accuracy: 62.0415%, 
 Time (train+val): 3699.9022s
Epoch : 001, Training: Loss: 0.6659, Accuracy: 61.7021%, 
		Validation : Loss : 0.6639, Accuracy: 62.0415%, 
 Time (train+val): 3691.0585s
Epoch : 002, Training: Loss: 0.6657, Accuracy: 61.7021%, 
		Validation : Loss : 0.6640, Accuracy: 62.0415%, 
 Time (train+val): 3685.7544s
Epoch : 003, Training: Loss: 0.6657, Accuracy: 61.7021%, 
		Validation : Loss : 0.6639, Accuracy: 62.0415%, 
 Time (train+val): 3651.2884s
Epoch : 004, Training: Loss: 0.6656, Accuracy: 61.7021%, 
		Validation : Loss : 0.6639, Accuracy: 62.0415%, 
 Time (train+val): 3632.3584s


In [ ]:
y_pred_list = []
y_test = []

alexnet.eval()
with torch.no_grad():
    for X_batch in test_data:
#         X_batch = X_batch.to(device)
        y_test_pred = alexnet(X_batch[0])
#         y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        labels = X_batch[1]
        y_test.append(labels) 
#         print(type(X_batch[0]))
#         break
        
        
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_test = [a.squeeze().tolist() for a in y_test]

In [ ]:
len(y_test[0])

In [ ]:
print(y_pred_list[0])

In [ ]:
metrics.confusion_matrix(y_test, y_pred_list)